In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# Import API key
from config import g_key

In [2]:
#store part 1 into df

In [3]:
# Create weather dataframe
weather_df= pd.read_csv('../WeatherPy/weather_output.csv')
weather_df.dropna(inplace = True) 
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Date,Wind Speed,Country
0,plettenberg bay,-34.0527,23.3716,55.51,77,30,1622488367,7.14,ZA
1,pangnirtung,66.1451,-65.7125,32.00,93,75,1622488276,11.50,CA
2,chuy,-33.6971,-53.4616,62.33,54,17,1622488368,15.35,UY
3,aykhal,66.0000,111.5000,52.21,51,100,1622488368,9.33,RU
4,bluff,-46.6000,168.3333,47.14,74,99,1622488368,8.50,NZ


In [4]:
humidity = weather_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = weather_df[["Lat", "Lng"]]

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
weather_df = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & (weather_df["Cloudiness"] == 0), :]
weather_df = weather_df.dropna(how='any')
weather_df.reset_index(inplace=True)
del weather_df['index']
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Date,Wind Speed,Country
0,rikitea,-23.1203,-134.9692,74.62,64,0,1622488373,14.94,PF
1,santiago del estero,-27.7951,-64.2615,74.41,26,0,1622488284,10.58,AR
2,guerrero negro,27.9769,-114.0611,70.47,50,0,1622488596,10.58,MX
3,antalaha,-14.9003,50.2788,71.11,84,0,1622488670,6.08,MG
4,mantua,45.1667,10.7833,72.55,61,0,1622488678,6.46,IT


In [7]:
hotellist = []

for i in range(len(weather_df)):
    lat =weather_df.loc[i]['Lat']
    lng =weather_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
weather_df["Hotel Name"] = hotellist
weather_df = weather_df.dropna(how='any')
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Date,Wind Speed,Country,Hotel Name
0,rikitea,-23.1203,-134.9692,74.62,64,0,1622488373,14.94,PF,Rikitea
1,santiago del estero,-27.7951,-64.2615,74.41,26,0,1622488284,10.58,AR,Santiago del Estero
2,guerrero negro,27.9769,-114.0611,70.47,50,0,1622488596,10.58,MX,Guerrero Negro
3,antalaha,-14.9003,50.2788,71.11,84,0,1622488670,6.08,MG,Antalaha
4,mantua,45.1667,10.7833,72.55,61,0,1622488678,6.46,IT,Mantua


In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in weather_df.iterrows()]
locations = weather_df[["Lat", "Lng"]]

In [9]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))